# WiDS Intro to ML

In [ ]:
from sklearn import datasets
import pandas as pd #Import the pandas library: Pandas is a Python library for data analysis.
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier # import the class DecisionTreeClassifier which implements the decision tree classifier

%matplotlib inline
import sys

## Data Source:
- Suppose we are working on a Spotify Song recommendation system, we want to predict if a customer will like a given song or not based on the song's features.

- Data Description:
> A dataset of 2017 songs with attributes from Spotify's API. Each song is labeled "1" meaning I like it and "0" for songs I don't like. Let's build a classifier that could predict whether or not I would like a song. 

> The original dataset can be found here: 
https://www.kaggle.com/datasets/geomack/spotifyclassification

<div>
<img src="https://github.com/Annie814/WiDS-ML-Workshop/blob/main/dataset-cover.jpeg?raw=1" width="600"/>
</div>

## Step 1: we will import the data using Panda's read_csv() function:
- In the bracket, you must include the file path of the data.
    - *Note: index_col=0, indicates what column of the csv to use as the indexes (row labels) of the dataframe. If the dataset doesn't have an id column, You generally don't have to specify that.*

In [ ]:
spotify_df = pd.read_csv("data.csv", index_col = 0)
spotify_df.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


Now we split the dataset into train and test sets:
> We divide our data into training and test set in the ratio 80:20 respectively (i.e if your input dataset has 100 rows, training data will be 80 rows randomly selected from the input, and test set will have the remaining 20 rows).

### Golden Rule of Machine Learning

Never mix training and test data together. Always isolate the test set, so that we can use the test set to make prediction and evaluate our model.
> You can think of `train_df` as the Practice exams & `test_df` as the FINAL exam.

Now lets look at some code:


In [ ]:
#train_df will contain 80% of the input data, test_df will contain 20% of the input data (flights_df)
train_df, test_df = train_test_split(spotify_df, test_size=0.2)

In [ ]:
# 1613/(1613 + 404) = 0.8
train_df.shape

(1613, 16)

In [ ]:
test_df.shape

(404, 16)

## Step 2: Clean data
Let's look at the dataset information to see if there are incomplete rows

Looking at `train_df.info()` gives us a rough idea about the features being numeric or textual and the number of `NaNs` in each feature:
- Luckily, we don't have any null values in our data, as we can see below:

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1613 entries, 1687 to 717
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      1613 non-null   float64
 1   danceability      1613 non-null   float64
 2   duration_ms       1613 non-null   int64  
 3   energy            1613 non-null   float64
 4   instrumentalness  1613 non-null   float64
 5   key               1613 non-null   int64  
 6   liveness          1613 non-null   float64
 7   loudness          1613 non-null   float64
 8   mode              1613 non-null   int64  
 9   speechiness       1613 non-null   float64
 10  tempo             1613 non-null   float64
 11  time_signature    1613 non-null   float64
 12  valence           1613 non-null   float64
 13  target            1613 non-null   int64  
 14  song_title        1613 non-null   object 
 15  artist            1613 non-null   object 
dtypes: float64(10), int64(4), object(2)
memo

In [ ]:
train_df.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
1687,0.000353,0.420,210240,0.929,0.000747,7,0.1220,-3.899,0,0.1210,127.204,4.0,0.318,0,Chop Suey!,System Of A Down
499,0.036500,0.829,336906,0.637,0.023300,5,0.2670,-9.481,0,0.0882,120.761,4.0,0.688,1,Changes,Crazy P
688,0.093800,0.874,281373,0.704,0.000000,9,0.4090,-6.270,1,0.1020,119.954,4.0,0.444,1,Shabba,A$AP Ferg
389,0.000239,0.329,255413,0.814,0.094400,7,0.0936,-6.005,1,0.0674,120.462,4.0,0.403,1,The Moment,Tame Impala
1391,0.289000,0.712,223333,0.625,0.000000,7,0.1270,-8.832,1,0.1490,100.004,4.0,0.718,0,Dangerous,Clazziquai Project


We can see that song title, artist and target are categorical variables. Therefore we need to transform these into numeric variables.

number dummy variables = # categories - 1

### How can we do this in python?
By Column transformer (will be introduced in the future), but if you are interested you could learn about it here: https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
<div>
<img src="https://github.com/Annie814/WiDS-ML-Workshop/blob/main/columntransformer.png?raw=1" width="800"/>
</div>

### For simplicity, We will remove the text features from the data and just understand the model using numeric features for now, Since textual features need some preprocessing.

In [ ]:
train_df = train_df.drop(columns=["song_title", "artist"])
test_df = test_df.drop(columns=["song_title", "artist"])

## Step 3: Train Model

### Now we are ready to train our model!
The next step is to build a model using a machine learning algorithm. There are a lot of algorithms out there. Each algorithm has its pros and cons in terms of the performance. For this workshop we will use a very simple algorithm called Decision Tree.
- We don't have to program this algorithm, it's actually already implemented for us in the library called scikit-learn (you will see the library imported the class DecisionTreeClassifier which implements the decision tree classifier).

### Here we are separating the target (AKA, response variables, y variable..) from the features (AKA, independent variables,predictors,x variables..)


In [ ]:
# features,target :will be usedFOR TRAINING, think about them as the Practise exams and their answer keys
X_train,y_train = train_df.drop(columns=["target"]), train_df['target']

# features,target :will be used FOR TESTING, think about them as the FINAL exams and  answer keys
X_test, y_test = test_df.drop(columns=["target"]), test_df['target']

In [ ]:
X_train

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
1687,0.000353,0.420,210240,0.929,0.000747,7,0.1220,-3.899,0,0.1210,127.204,4.0,0.318
499,0.036500,0.829,336906,0.637,0.023300,5,0.2670,-9.481,0,0.0882,120.761,4.0,0.688
688,0.093800,0.874,281373,0.704,0.000000,9,0.4090,-6.270,1,0.1020,119.954,4.0,0.444
389,0.000239,0.329,255413,0.814,0.094400,7,0.0936,-6.005,1,0.0674,120.462,4.0,0.403
1391,0.289000,0.712,223333,0.625,0.000000,7,0.1270,-8.832,1,0.1490,100.004,4.0,0.718
...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,0.006490,0.805,213280,0.596,0.007720,7,0.2990,-6.007,1,0.1190,81.004,4.0,0.753
1327,0.000046,0.261,149278,0.863,0.000007,11,0.3290,-7.199,1,0.0721,108.347,4.0,0.316
1875,0.081600,0.611,152869,0.487,0.116000,2,0.1110,-10.629,1,0.0275,105.034,3.0,0.667
1724,0.067000,0.829,243159,0.652,0.000000,7,0.0812,-8.693,0,0.1080,99.581,4.0,0.717


In [ ]:
y_train

1687    0
499     1
688     1
389     1
1391    0
       ..
463     1
1327    0
1875    0
1724    0
717     1
Name: target, Length: 1613, dtype: int64

Let's talk about Decision Tree. Our goal here is to determine weather the customer likes the song or not. We can use Decision Tree algorithm here for creating the model for classification problems like this one.

### What is a Decision Tree?

- classification model that predicts value of target based on learning simple "rules" that it obtained from the data pattern.
Let us consider the following data:
<div>
<img src="https://github.com/Annie814/WiDS-ML-Workshop/blob/main/img.png?raw=1" width="1000"/>
</div>

This is one possible tree for this problem.
<div>
<img src="https://github.com/Annie814/WiDS-ML-Workshop/blob/main/DecisionTree.png?raw=1" width="1000"/>
</div>

**Pros of Decision Tree Classifier:**

    - simple to understand
    - easy to visualize
    - generate understandable rules.
    - perform classification without requiring much computation.

**Cons:**

    - less appropriate for estimation tasks where the goal is to predict the value of a continuous attribute.
    - prone to errors in classification problems with many class and a relatively small number of training examples.

### How can we build a decision tree model in python?

Now lets fit the model:


In [ ]:
# We need to create an object, let's call it model, and set it to a new instance of Decision Tree Classifier.
model = DecisionTreeClassifier()
# next we need to train it, so it learns the patterns of the INPUT train-set & OUTPUT train-set. Recall the Golden Rule of ML.
# The input is X_train, the output is y_train
model.fit(X_train, y_train)
# The output is just a visual for the model object you just fitted/trained.

DecisionTreeClassifier()

## Step 4: Make Predictions
After fitting the model, we can predict on the testing set, and also evaluate our decision tree by calculating its accuracy.

In [ ]:
predict = model.predict(X_test)
predict

array([0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,

In [ ]:
# combining the predicted value with the test data set:
pred_result = test_df
pred_result['predicted'] = predict.tolist()

In [ ]:
pred_result

## Step 5: Evaluate the model and make improvement on its performance (Accuracy)

In [ ]:
accuracy = accuracy_score(y_test, predict)
accuracy

> The test accuracy score is one of the metrics to see how our model generalize to unseen data (This is the key for Machine Learning: Our model should generalize well for new observations). We want our model to have high accuracy when predicting the test set, so we have more confident on the model accuracy after it's deployed to real world.

### Improving the model accuracy by Hyperparameter tuning:
Did some magic here to tune the parameter that goes into our DecisionTree Algorithm:

In [ ]:
from sklearn.model_selection import cross_validate
import numpy as np
results_dict = {
    "max_depth": [],
    "mean_train_score": [],
    "mean_cv_score": []
}

for depth in range(1, 26):
    model = DecisionTreeClassifier(max_depth=depth)
    cv_score = cross_validate(model, X_train, y_train, cv=10, return_train_score=True)
    results_dict["max_depth"].append(depth)
    results_dict["mean_cv_score"].append( np.mean(cv_score["test_score"]))
    results_dict["mean_train_score"].append( np.mean(cv_score["train_score"]))

result_df = pd.DataFrame(results_dict)
result_df

In [ ]:
optimized_depth = result_df[result_df.mean_cv_score == result_df.mean_cv_score.max()].iloc[0,0]
model_optimized = DecisionTreeClassifier(max_depth = optimized_depth)
model_optimized.fit(X_train, y_train)

In [ ]:
optimized_depth

In [ ]:
predict = model_optimized.predict(X_test)

accuracy = accuracy_score(y_test, predict)

accuracy

In reality most classifiers will never have 100% accuracy. And indeed in our case it doesn't reach a 100% accuracy. This is just a quick demo to show you the Step 5 where further improvements on the model can be done using Hyperparameter tuning.

### Exercis for you: Let's Dive into Some Python Codes Now

Now we are building a model on the iris flower dataset. Start by importing the datasets library from `sklearn`, and load the iris data set.

In [ ]:
#Load dataset + create a dataframe of this iris dataset. (you can ignore this step)
iris =  datasets.load_iris()
data = pd.DataFrame({
    'sepal_length': iris.data[:,0],
    'sepal_width': iris.data[:,1],
    'petal_length': iris.data[:,2],
    'petal_width': iris.data[:,3],
    'species': iris.target
})

data.head(5)

Let's print the target and feature variable names just to make sure that we are using the right dataset

In [ ]:
print(iris.target_names)
print(iris.feature_names)

### Now Please complete the `#TODO`s:

We use the `train_test_split` function to split variables into train and test set (Let's take 75% to training and 25% to testing), and train the model on the train set and perform predictions on the test set.

In [ ]:
train_df, test_df = train_test_split(#TODO)

Since the species of iris flower is what we are interested in classifying, we first separate columns accordingly into dependent and independent variables.
Steps as follow:

In [ ]:
# Seperate cols to dependent and independent variables
X_train, y_train = train_df.drop(columns=["species"]), train_df["species"]
X_test, y_test = #TODO

In [ ]:
# create the classifier
model = #TODO
model.fit(#TODO)
# make the prediction using the Decision Tree model with the test x values( hint: X_test)
y_pred = model.predict(#TODO)

We now examine the Decision tree model's accuracy using the actual y (species) value and the predicted values given by the model.

In [ ]:
print("The accuracy for Decision Tree model is: ", round(accuracy_score(y_test, y_pred),4))

And we say the accuracy is pretty high for such model! To make a prediction on a single observation, we can also use the `predict()` function.
For example:  
    - sepal length = 3  
    - sepal width = 6  
    - petal length = 6  
    - petal width = 4  
Now we can predict which type of the iris flower it is as below.

In [ ]:
model.predict([[3,6,6,4]])

Here, the output is 2, which indicates an iris type of Virginica.

<div>
<img src="https://github.com/Annie814/WiDS-ML-Workshop/blob/main/survey.jpg?raw=1" width="500"/>
</div>
Congratulation! You have made it so far and know what a typical Decision Tree classifier in python looks like. Please spend a minute to fill out this post workshop survey. Your feedback means a lot to us.

### references
https://www.youtube.com/watch?v=7eh4d6sabA0&ab_channel=ProgrammingwithMosh
https://github.com/hmartin11/USS_ML_Workshop
https://www.edureka.co/blog/decision-trees/

